In [1]:
import pandas as pd

In [2]:
import warnings
warnings.filterwarnings ("ignore")

In [3]:
# Función para cargar CSV
def cargar_csv(ruta_archivo):
    return pd.read_csv(ruta_archivo)

In [4]:
# Archivos y rutas
archivos = {
    'Sales_final_2016': r'C:\Users\silvina\Downloads\HENRY\PF Inventory\SalesFINAL12312016.csv',
    'Purchase_final_2016': r'C:\Users\silvina\Downloads\HENRY\PF Inventory\PurchasesFINAL12312016.csv',
    'Invoice_purchases_2016': r'C:\Users\silvina\Downloads\HENRY\PF Inventory\InvoicePurchases12312016.csv',
    'Purchase_Prices_2017': r'C:\Users\silvina\Downloads\HENRY\PF Inventory\2017PurchasePricesDec.csv',
    'Beg_inv_final_2016': r'C:\Users\silvina\Downloads\HENRY\PF Inventory\BegInvFINAL12312016.csv',
    'End_inv_final_2016': r'C:\Users\silvina\Downloads\HENRY\PF Inventory\EndInvFINAL12312016.csv'
}

In [5]:
# Cargar todas las tablas
tablas = {nombre: cargar_csv(ruta) for nombre, ruta in archivos.items()}

In [6]:
# Función para convertir columnas de fecha a datetime automáticamente
def convertir_a_datetime(df):
    for columna in df.columns:
        if df[columna].dtype == 'object' and any(keyword in columna.lower() for keyword in ['fecha', 'date', 'timestamp']):
            try:
                df[columna] = pd.to_datetime(df[columna], errors='coerce')
            except Exception as e:
                print(f"Error al convertir la columna {columna} a datetime: {e}")
    return df


In [7]:
# Aplicar la conversión de fechas a todas las tablas
tablas_con_fechas_convertidas = {nombre: convertir_a_datetime(df) for nombre, df in tablas.items()}

In [8]:
# Verificar los tipos de datos después de la conversión
for nombre, df in tablas_con_fechas_convertidas.items():
    print(f"Tipos de datos para {nombre}:")
    print(df.dtypes)
    print()

Tipos de datos para Sales_final_2016:
InventoryId               object
Store                      int64
Brand                      int64
Description               object
Size                      object
SalesQuantity              int64
SalesDollars             float64
SalesPrice               float64
SalesDate         datetime64[ns]
Volume                     int64
Classification             int64
ExciseTax                float64
VendorNo                   int64
VendorName                object
dtype: object

Tipos de datos para Purchase_final_2016:
InventoryId               object
Store                      int64
Brand                      int64
Description               object
Size                      object
VendorNumber               int64
VendorName                object
PONumber                   int64
PODate            datetime64[ns]
ReceivingDate     datetime64[ns]
InvoiceDate       datetime64[ns]
PayDate           datetime64[ns]
PurchasePrice            float64
Quantity       

In [9]:
def limpiar_nulos(df):
    # Eliminar filas con cualquier valor nulo
    df = df.dropna()
# O también, si prefieres rellenar los valores nulos, puedes usar algo como:
# Rellenar valores nulos en columnas numéricas con la media
    for columna in df.select_dtypes(include=['float64', 'int64']).columns:
        df[columna] = df[columna].fillna(df[columna].mean())
# Rellenar valores nulos en columnas categóricas con la moda (valor más frecuente)
    for columna in df.select_dtypes(include=['object']).columns:
        df[columna] = df[columna].fillna(df[columna].mode()[0])

    return df

In [10]:
# Aplicar limpieza de nulos a todas las tablas
tablas_sin_nulos = {nombre: limpiar_nulos(df) for nombre, df in tablas_con_fechas_convertidas.items()}

In [11]:
# Verificar que no haya más valores nulos
for nombre, df in tablas_sin_nulos.items():
    print(f"Conteo de valores nulos para {nombre}:")
    print(df.isnull().sum())  # Muestra la cantidad de nulos por columna
    print(df.head())  # Muestra las primeras filas del DataFrame
    print()

Conteo de valores nulos para Sales_final_2016:
InventoryId       0
Store             0
Brand             0
Description       0
Size              0
SalesQuantity     0
SalesDollars      0
SalesPrice        0
SalesDate         0
Volume            0
Classification    0
ExciseTax         0
VendorNo          0
VendorName        0
dtype: int64
           InventoryId  Store  Brand                 Description        Size  \
0  1_HARDERSFIELD_1004      1   1004  Jim Beam w/2 Rocks Glasses       750mL   
1  1_HARDERSFIELD_1004      1   1004  Jim Beam w/2 Rocks Glasses       750mL   
2  1_HARDERSFIELD_1004      1   1004  Jim Beam w/2 Rocks Glasses       750mL   
3  1_HARDERSFIELD_1004      1   1004  Jim Beam w/2 Rocks Glasses       750mL   
4  1_HARDERSFIELD_1005      1   1005     Maker's Mark Combo Pack  375mL 2 Pk   

   SalesQuantity  SalesDollars  SalesPrice  SalesDate  Volume  Classification  \
0              1         16.49       16.49 2016-01-01     750               1   
1              2 

In [12]:
pip install sqlalchemy 

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [13]:
pip install pyodbc

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [15]:
df.to_csv('any_manufacturing.csv', index=False)

In [16]:
pwd

'c:\\Users\\silvina\\Downloads\\HENRY\\PF Inventory'

In [17]:
#Para SQL Server
from sqlalchemy import create_engine # Biblioteca que yo uso para conectarme con base de datos locales
from sqlalchemy.orm import sessionmaker
from sqlalchemy.exc import SQLAlchemyError
import pyodbc
import pandas as pd
import numpy as np
import os


In [19]:
#optativo pero recomendable por seguridad
from dotenv import load_dotenv 


In [20]:
# Cargar las variables de entorno desde el archivo .env
load_dotenv()

False

In [21]:
# Obtener las variables de entorno
password = os.getenv("DB_PASSWORD")

In [22]:
# Configuración de la conexión
user = "DESKTOP-ONSUG5I\silvina" #tu usuario SQL Server
password = password # Reemplazar con el password de tu motor local para no exponerlo si se sube a un repositorio publico
host = "localhost"
port = "1433" # Puerto por defecto de SQL Server (Verificar)
database = "basedatos_proyectofinal_prueba" # Nombre de la base de datos

In [23]:
# Crear la URI de conexión para SQL Server
connection_uri = f'mssql + pyodbc://{user}:{password}@{host}:{port}/{database}??Driver= ODBC+Driver+17+for+SQL+Server'

In [ ]:
# Crear el motor de conexión
engine = create_engine(connection_uri)

In [ ]:
# Verifica que la conexión sea exitosa
if engine.connect():
    print("Conexión exitosa a la base de datos.")
else:
    print("Error en la conexión.")
